In [ ]:
% matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from IPython.display import display, Markdown, Latex

In [ ]:
def zr(x):
    # x-sorted
    return (x[0] + x[-1]) / 2


def ztr(x):
    a = 0.05
    n = len(x)
    r = round(a * n)
    return np.sum(x[r:n - r - 1]) / (n - 2 * r)


def zq(x):
    n = len(x)
    if n // 4 == 0:
        lq = x[-1]
    else:
        lq = x[round(n / 4)]
    if (3 * n) // 4 == 0:
        uq = x[3 * n / 4]
    else:
        uq = x[round(3 * n / 4)]

    return (lq + uq) / 2


def contaminated(size):
    samples=[]
    for i in range(size):
        if 0.9<np.random.uniform():
            samples.append(np.random.standard_cauchy())
        else:
            samples.append(np.random.normal())
    return samples



def evaluate(distr_name, distribution, m, sizes):
    display(Markdown('# '+distr_name))
    for h in sizes:
        display(Markdown('## {:3d} samples'.format(h)))
        measures = {'mean': [], 'med': [], 'zr': [], 'ztr': [], 'zq': []}
        
        shoot = np.random.randint(0, m)
        cycol = cycle('kbgrcm')
        for i in range(1, m):
            samples = distribution(size=h)
            samples.sort()
            measures['mean'].append(np.sum(samples) / len(samples))
            measures['med'].append(np.median(samples))
            measures['zr'].append(zr(samples))
            measures['ztr'].append(ztr(samples))
            measures['zq'].append(zq(samples))
            if shoot == i:
                n, bins, patches = plt.hist(samples, histtype='bar', bins=h, facecolor=next(cycol),
                                            alpha=0.75)
                plt.title('{} - {:d} samples'.format(distr_name, h))
                plt.tight_layout()
                plt.grid(True)

                for (name, measure) in measures.items():
                    plt.axvline(x=measure[-1], label='{:<5} {:.4f}'.format(name, measure[-1]), c=next(cycol))
                plt.legend()
                #plt.savefig('out/{}_{:d}.png'.format(distr_name, h), bbox_inches='tight')
                plt.show()

        print(r'{:>10}: {:>10}: {:>10}'.format('Measure','mean','variance'))
        for (name, measure) in measures.items():
            mean=np.sum(measure) / len(measure)
            variance=np.sum([x**2 for x in measure])/len(measure)-(np.sum(measure) / len(measure))**2
            print(r'{:>10}: {: 10f}: {: 10f}'.format(name, mean,variance))
    display(Markdown("***"))


In [ ]:
M = 1000
H = (20, 50, 100)
#np.random.seed(1658090)

evaluate('Standart normal', np.random.normal, M, H)
evaluate('Uniform', (lambda size: np.random.uniform(-.5, .5, size)), M, H)
evaluate('Laplace', np.random.laplace, M, H)
evaluate('Cauchy', np.random.standard_cauchy, M, H)
evaluate('CCN', contaminated, M, H)